In [1]:
#imports

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import os
import cv2

The URL is from NYC DOT Real-Time 
Traffic Information which allows access to the public to ~900 CCTV cameras of New York City Roads. This code Is only scraps to images provided from one camera that being from camera WBB SIR -2 @ Manhattan Mid Span that is west facing. The end goal of this project is to observe more bridge and/ or tunnel cameras to see the volume of traffic going in and out of different boroughs to see what areas are most populated at what hour. To achieve this goal the script will need to run at different times of the day and week to see the difference by workday. potentially be used during different times of the year to see if there are any seasonal changes.


In [3]:
# URL of the webpage containing the image
url = "https://webcams.nyctmc.org/api/cameras/368290bb-cebf-4845-8644-92382dcccd96/image"

In [4]:
#Save location of the images as jp's
output_directory = os.path.expanduser("~/Desktop/NYC_pic_test/Manhattan_Mid_Span_West_facing/Images")

In [5]:
os.makedirs(output_directory, exist_ok=True)

In [6]:
try:
    while True:
        try:
            # Fetch the image data
            response = requests.get(url)
            image_data = response.content
    
            # Save the image data to a file
            image_filename = os.path.join(output_directory, f"captured_image_{int(time.time())}.jpg")
            with open(image_filename, "wb") as f:
                f.write(image_data)
    
            s = image_filename
            # Find the last occurrence of '/'
            last_slash_index = s.rfind('/')
            
            # Slice the string to get everything after the last '/'
            result = s[last_slash_index+1:]
            location = s[ :last_slash_index]
            
            print(f"Image saved as {result}")
    
            #print(f"Image located at {location}")
    
            # Wait for 1 seconds before capturing the next image
            time.sleep(1)
    
        except Exception as e:
            print(f"Error: {e}")
            # Handle any exceptions (e.g., network issues)

except KeyboardInterrupt:
    print("Program terminated by user.")


Image saved as Images\captured_image_1712065961.jpg
Image saved as Images\captured_image_1712065963.jpg
Image saved as Images\captured_image_1712065965.jpg
Image saved as Images\captured_image_1712065967.jpg
Image saved as Images\captured_image_1712065969.jpg
Image saved as Images\captured_image_1712065970.jpg
Image saved as Images\captured_image_1712065972.jpg
Image saved as Images\captured_image_1712065974.jpg
Image saved as Images\captured_image_1712065976.jpg
Image saved as Images\captured_image_1712065978.jpg
Image saved as Images\captured_image_1712065980.jpg
Image saved as Images\captured_image_1712065982.jpg
Image saved as Images\captured_image_1712065984.jpg
Image saved as Images\captured_image_1712065986.jpg
Image saved as Images\captured_image_1712065987.jpg
Image saved as Images\captured_image_1712065989.jpg
Program terminated by user.


In [7]:
# Specify the directory containing your JPG files
image_folder = 'C:/Users/Jared Goroski/Desktop/NYC_pic_test/Manhattan_Mid_Span_West_facing/Images'
video_name = '~/NYC_pic_test/Manhattan_Mid_Span_West_facing/Video/my_video.mp4'


# Get a list of all JPG files in the specified folder
images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]

# Read the first image to get its dimensions
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# Create a VideoWriter object to save the video
video = cv2.VideoWriter(video_name, 0, 1, (width, height))

# Write each image to the video
for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

# Release the video writer
video.release()


s = video_name
# Find the last occurrence of '/'
last_slash_index = s.rfind('/')

# Slice the string to get everything after the last '/'
result = s[last_slash_index+1:]
location = s[ :last_slash_index]


#prints the name of the video if the script works as it should
print(f"Video saved as {result}")

#use if need to print save location of video
#print(f"Video saved at {location}")

Video saved as my_video.mp4


In [ ]:
# Capture video from file
cap = cv2.VideoCapture('~/NYC_pic_test/Manhattan_Mid_Span_West_facing/Video/my_video.mp4')

# Minimum rectangle width and height
min_width_react=80
min_hieght_react=80

# Line position for counting
count_line_postion = 550

# Initialize Substructor
algo = cv2.bgsegm.createBackgroundSubtractorMOG()

def center_handle(x,y,w,h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x + x1
    cy = y + y1
    return cx, cy

detect = []

# Infinite loop to process video frames
while True:
    ret,frame1=cap.read()
    grey = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey,(3,3),5)
    #applying on each frame
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub,np.ones((5,5)))
    kernal = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,())
    dilatada = cv2.morphologyEX(dilat, cv2.MORPH_CLOSE, kernal)
    dilatada = cv2.morphologyEX(dilatada, cv2.MORPH_CLOSE, kernal)
    counterShape = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    h = counterShape

    # Draw a line where the counting takes place
    cv2.line(frame1,(25,count_line_position),(1200,count_line_position),(255,127,0),3)

    # Loop over each contour found
    for (i,c) in enumerate(counterShape):
        (x,y,w,h) = cv2.boundingRect(c)   # Get bounding box for each contour
        validate_counter = (w >= min_width_react) and (h >= min_hieght_react)  # Validate contour by size
        if not validate_counter:
            continue # Skip small contours

        # Draw rectangle around valid contours
        cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,255,0),2)

         # Calculate center of the contour and add to detect list
        center = center_handle(w,y,w,h)
        detect.append(center)
        
        # Draw center of the contour
        cv2.circle(frame1,center,4, (0,0,255),-1)


        for (x,y) in detect:
            if x < (count_line_postion + offset) and y < (count_line_postion + offset):
                counter += 1
            cv2.line(frame1,(25,count_line_postion),(1200,count_line_postion),(0,127,255),3)
            detect.remove(x,y)
            
            print("Vehicle Counter:" + str(counter))

    # Display vehicle count on the frame
    cv2.putText(frame1,"VEHICLE COUNTER :" + str(counter),(450,70),cv2.FONT_HERSHEY_SIMPLEX)

    # Show the frame
    cv2.imshow('Video Original', frame1)

    # Break the loop when 'Enter' key is pressed
    if cv2.waitKey(1) == 13:
        break

# Clean up: close all windows and release video capture object
cv2.destroyAllWindows()
cap.release()
    